In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import time

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [29]:
from tensorflow.keras.utils import CustomObjectScope, get_custom_objects
import functools
import keras
import sys
sys.path.append('../input/')
sys.path.append('../input/bvvkerasapp/')

In [30]:
pip install ../input/tfatfa2/tensorflow_addons-0.10.0-cp36-cp36m-manylinux2010_x86_64.whl -f ./ --no-index

ERROR: tensorflow_addons-0.10.0-cp36-cp36m-manylinux2010_x86_64.whl is not a supported wheel on this platform.
Note: you may need to restart the kernel to use updated packages.


In [31]:
from tensorflow_addons.metrics import  CohenKappa
kappa_keras = CohenKappa(num_classes=6,weightage='quadratic')

In [32]:
# from bvveffn9 import model

In [33]:
# def init_keras_custom_objects():
#     custom_objects = {
#         'FixedDropout': inject_keras_modules(model.get_dropout)()
#     }

#     get_custom_objects().update(custom_objects)

In [34]:
# def inject_keras_modules(func):
#     @functools.wraps(func)
#     def wrapper(*args, **kwargs):
#         kwargs['backend'] = keras.backend
#         kwargs['layers'] = keras.layers
#         kwargs['models'] = keras.models
#         kwargs['utils'] = keras.utils
#         return func(*args, **kwargs)
#     return wrapper

In [35]:
# init_keras_custom_objects()

In [36]:
import skimage.io
import cv2
import openslide
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json, load_model



In [37]:
test_datagen = ImageDataGenerator(rescale = 1/255,
                                    shear_range=0.05,
                                    zoom_range=0.05,
                                    horizontal_flip=True,
                                    rotation_range=20.,
                                    fill_mode='reflect',
                                    width_shift_range = 0.1, 
                                    height_shift_range = 0.1)

In [38]:
#os.listdir('../input/panda-model-1')

In [54]:
#folders:
base_path = "../input"

#for debug:
from_fold = '../input/prostate-cancer-grade-assessment/train_images' 
test_labels = pd.read_csv('../input/prostate-cancer-grade-assessment/train.csv')
labels_perm = test_labels.sample(frac = 1, random_state = 137)
test_labels = labels_perm[9400:]


#for submission
# from_fold = '../input/prostate-cancer-grade-assessment/test_images' 
# test_labels = pd.read_csv('../input/prostate-cancer-grade-assessment/test.csv')


In [40]:
# model_path ='bvvmodel' #this is a name of the folder, where we upload the model
# model_name = 'model_panda_check.12.h5'


In [41]:
model_path ='resnetkappa' #this is a name of the folder, where we upload the model
model_name = 'model_panda_check.19.h5'

In [42]:
#all constatnts
# dim_y = 320
# dim_x = 320
# tta_steps = 10


In [43]:
#load model:
new_model = load_model(os.path.join(base_path, model_path, model_name))

In [44]:
#proc_func input parameters:
# arg_to_proc_func = {
#                 'sz':80,
#                 'N':16} 
arg_to_proc_func = {
                'sz':128,
                'N':16}

In [56]:
def prepare_test_data(proc_func):

    def wrapper(df_name = test_labels[:10],
                       zoom = 2,
                       **kw):

        """
        This is based on 'eliminate_white_direct'
        """
        test_data = []
        images_names = []
        
        for i,row in enumerate(df_name.iterrows()):        
            image_id = row[1]['image_id']
            provider = row[1]['data_provider'] 
            images_names.append(image_id)
            test_file = os.path.join(from_fold, image_id + '.tiff')
            biopsy = openslide.OpenSlide(test_file)            
            yyy = biopsy.level_dimensions[zoom]
            biopsy = np.array(biopsy.read_region((0,0), zoom, yyy).convert('RGB'))

            #central block:
            try:
                data_new = proc_func(biopsy, provider, **kw)
            except:
                data_new = biopsy
            #end central block
            
            biopsy = data_new #cv2.resize(data_new, (dim_y, dim_x))
            
            try:
                del data_new
            except:
                pass
           
            test_data.append(biopsy)
             
            del biopsy
    
        return np.array(images_names), np.array(test_data) #wrapper end
    
    return wrapper #decorator end

In [57]:
@prepare_test_data
def img_func(img, provider, **kw):        
    sz=kw.get('sz') if kw.get('sz') else 32    
    N = kw.get('N') if kw.get('N') else 16
    final_dim = kw.get('final_dim') if kw.get('final_dim') else N**0.5 
    result = []
    shape = img.shape
    #if shape[0]<shape[1]: 
    #    img = np.rot90(img)
    #    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    a, b, c, d = img.shape
    data_new = (img.reshape(int(final_dim), int(final_dim),\
                                                    b, c, d).swapaxes(1,2).reshape(b*int(final_dim),\
                                                    c*int(final_dim), d))
    data_new = cv2.bitwise_not(data_new)

    return data_new

#substitute for appropriate prcocessing function

In [47]:
def make_final_labels(from_model_preds):
    return from_model_preds.argmax(axis=-1)

In [48]:
def make_submission_df(id_array, test_labels):
    d2 = {'image_id': id_array, 'isup_grade': test_labels}
    new_sample = pd.DataFrame(data=d2)
    return new_sample

In [49]:
# def prepare_predict_arr_on_tta(ttagen, steps_, tta_steps = tta_steps):    
#     pred_new_arr = []
#     for i in range(tta_steps):
#         pred_new = new_model.predict_generator(ttagen, steps = steps_)
#         pred_new_arr.append(pred_new)
#     pred_all = np.mean(pred_new_arr, axis=0)
    
#     return pred_all

In [50]:
def prepare_predict_arr_on_tta(ttagen, steps_): 
    return new_model.predict_generator(ttagen, steps = steps_)

In [60]:
if os.path.exists(from_fold):
    #list of test files:
    test_files = os.listdir(from_fold)
    
    #prepare input data for the model:
    test_files_id_array, test_file_data_array = img_func(**arg_to_proc_func)
    
    #data generator:
    testgen = test_datagen.flow(
                            x=test_file_data_array,
                            y=None, 
                            batch_size=1,
                            shuffle = False,
                            )
    
    #predict with the model:
    #pred_all = prepare_predict_arr_on_tta(ttagen = testgen, steps_ = len(test_files_id_array))
    print('cow')
    pred_all = new_model.predict_generator(testgen, steps = len(test_files_id_array))
    print('rabbit')
    #make final predictions:
    final_test_labels = make_final_labels(pred_all)
    
    #make submission:
    submission = make_submission_df(test_files_id_array, final_test_labels)
    submission['isup_grade'] = submission['isup_grade'].astype(int)
    submission.to_csv('submission.csv', index=False)

else: #test data are covered
    sub = pd.read_csv("/kaggle/input/prostate-cancer-grade-assessment/sample_submission.csv")
    sub.to_csv('submission.csv', index=False)
    print('no submission data')

cow
rabbit


In [61]:
submission

,image_id,isup_grade
0,d3a5775345ddc4976944ff1941709ef0,0
1,e20c73041c5c5ac83c20a7c669f46e9a,0
2,8f8d20fc6e9b6aa3b145e359e104eac4,0
3,6ac2095c7968fa30ab6c5dc7d1ed5121,0
4,389758c9e3aad00e774d581150f372db,0
5,66f04788a17915feba394290384e13a4,0
6,0abc61a1bc7abe47b9e44a9f69979eb0,0
7,fd8928cd1f06f33ff00e32f19c58abd3,0
8,dfae643568b01d1f4ad5d86bac2f7994,0
9,fb80c40073a4016bf384f408ec048f55,0
